In [0]:
from __future__ import absolute_import, division, print_function

import copy
import json
import math

import six
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss

# 1. GELU

* x:

* torch.erf: 

In [0]:
def gelu(x):
    """Implementation of the gelu activation function.
        For information: OpenAI GPT's gelu is slightly different (and gives slightly different results):
        0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))
    """
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))

# 2. BERT Config

Guarda la configuracion del modelo BERT

* vocab_size: Tamaño del vocabulario de inputs_ids
* hidden_size: Tamaño de las capas del codificador y la capa de pool
* num_hidden_layers: Número de capas ocultas en el codificador de Transformer
* num_attention_heads: Número de attention heads para cada capa de atención en el codificador Transformer
* intermediate_size: Tamaño de la capa intermedia en el codificador de Transformer
* hidden_act: Función de activación no lineal del codificador y el pool
* hidden_dropout_prob: Probabilidad de dropout de todas las capas completamente conectadas en los embeddings, el codificador y el pool
* attention_probs_dropout_prob: Probabilidad de dropout para las probabilidades de atencion
* max_position_embeddings: Longitud de la secuencia máxima que se puede introducir al modelo
* type_vocab_size: Tamaño del vocabulario de token_type_ids
* initializer_range:  Valor de sttdev para truncated_normal_initializer que inicializa todas las matrices de pesos

In [0]:
class BertConfig(object):
    def __init__(self,
                vocab_size,
                hidden_size=768,
                num_hidden_layers=12,
                num_attention_heads=12,
                intermediate_size=3072,
                hidden_act="gelu",
                hidden_dropout_prob=0.1,
                attention_probs_dropout_prob=0.1,
                max_position_embeddings=512,
                type_vocab_size=16,
                initializer_range=0.02):
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.num_hidden_layers = num_hidden_layers
        self.num_attention_heads = num_attention_heads
        self.hidden_act = hidden_act
        self.intermediate_size = intermediate_size
        self.hidden_dropout_prob = hidden_dropout_prob
        self.attention_probs_dropout_prob = attention_probs_dropout_prob
        self.max_position_embeddings = max_position_embeddings
        self.type_vocab_size = type_vocab_size
        self.initializer_range = initializer_range

# 2.1. From dict

Construye un objeto BertConfig a partir de un diccionario de Python

* json_object: Objeto con la informacion en un diccionario de un archivo json del dataset

* config: Objeto BertConfig creado a partir de un diccionario
* six.iteritems: Compatibilidad para recorrer diccionarios de Python 2 y 3
* `config.__dict__[key]`: En el diccionario por defecto del objeto config se crea una entrada con el par (key, value) 

In [0]:
    @classmethod
    def from_dict(cls, json_object):
        config = BertConfig(vocab_size=None)
        for (key, value) in six.iteritems(json_object):
            config.__dict__[key] = value
        return config

# 2.2. From json file

Construye un objeto BertConfig a partir de un archivo json

* json_file: Path del archivo json

* text: El texto que el archvo json contiene
* json.loads: Transforma text en un objeto json

In [0]:
    @classmethod
    def from_json_file(cls, json_file):
        with open(json_file, "r") as reader:
            text = reader.read()
        return cls.from_dict(json.loads(text))

# 2.3. To dict

Exporta el diccionario por defecto de un objeto BertConfig

* copy.deepcopy: Crea una copia que incluye los elementos anidados
* output: El diccionario por defecto de un objeto BertConfig

In [0]:
    def to_dict(self):
        output = copy.deepcopy(self.__dict__)
        return output

# 2.4. To json string

Exporta el diccionario por defecto de un objeto BertConfig a un string con el formato de un archivo json

* json.dumps: Transforma un diccionario al formato de un archivo json

In [0]:
    def to_json_string(self):
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n"

# 3. BERT layer norm

Capa de normalizacion al estilo de TF<br>
La normalizacion ajusta valores en diferentes escalas respecto a una escala comun<br>
La normalizacion de capas acelera el proceso de entrenamiento (Ba et al.)

* variance_epsilon: Valor que ajusta la varianza para calcular la desviacion estandar
* gamma: Vector de valores para ajustar la norma
* beta: Vector de valores para ajustar la norma

* nn.Module: Modulos de pytorch con operaciones basicas de redes neuronales
* config: Objeto con los parametros de configuracion del modelo BERT
* hidden_size: Tamaño de las capas del codificador y la capa de pool
* torch.ones: Crea un vector de unos
* torch.zeros: Crea un vector de ceros

In [0]:
class BERTLayerNorm(nn.Module):
    def __init__(self, config, variance_epsilon=1e-12):
        super(BERTLayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(config.hidden_size))
        self.beta = nn.Parameter(torch.zeros(config.hidden_size))
        self.variance_epsilon = variance_epsilon

# 3.1. Forward

# $ Norm(x) = \frac{x - Mean}{Standard Deviation} $
# $ Standard Deviation = \sqrt{Variance} $

<br>

La normalizacion ajusta valores en diferentes escalas respecto a una escala comun<br>
La normalizacion de capas acelera el proceso de entrenamiento (Ba et al.)

* variance_epsilon: Valor que ajusta la varianza para calcular la desviacion estandar
* gamma: Vector de valores para ajustar la norma
* beta: Vector de valores para ajustar la norma

https://stackoverflow.com/questions/50973995/layer-normalization-and-how-it-works-tensorflow/52205057

# $ Norm(x) * gamma + beta $
Se multiplica cada dimension de $Norm(x)$ con $gamma$ y luego se le suma $beta$.<br>
$gamma=1$ y $beta=0$ para ignorar este paso.

<br>

* x: Valores de entrada
* u: Media de x
* s: Varianza de x

In [0]:
    def forward(self, x):
        u = x.mean(-1, keepdim=True)
        s = (x - u).pow(2).mean(-1, keepdim=True)
        x = (x - u) / torch.sqrt(s + self.variance_epsilon)
        return self.gamma * x + self.beta

# 4. BERT Embeddings

* config: Objeto con los parametros de configuracion del modelo BERT
* vocab_size: Tamaño del vocabulario de inputs_ids
* max_position_embeddings: Longitud de la secuencia máxima que se puede introducir al modelo
* type_vocab_size: Tamaño del vocabulario de token_type_ids
* hidden_size: Tamaño de las capas del codificador y la capa de pool


* self.word_embeddings: Variable para los embeddings del vocabulario de embeddings
* self.position_embeddings: Variable para los embeddings de la secuencia de entrada
* self.token_type_embeddings: Variable para guardar el vector que contiene el tipo del token


* self.LayerNorm: Se le añade una capa de normalizacion (Seccion 3)
* self.dropout: Dropout de la capa

In [0]:
class BERTEmbeddings(nn.Module):
    def __init__(self, config):
        super(BERTEmbeddings, self).__init__()
        """Construct the embedding module from word, position and token_type embeddings.
        """
        self.word_embeddings = nn.Embedding(config.vocab_size, config.hidden_size) # nn.Embedding(vocab_size, vector_size)
        self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size)
        self.token_type_embeddings = nn.Embedding(config.type_vocab_size, config.hidden_size)

        # self.LayerNorm is not snake-cased to stick with TensorFlow model variable name and be able to load
        # any TensorFlow checkpoint file
        self.LayerNorm = BERTLayerNorm(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

# 4.1. Forward

<img src=https://cdn-images-1.medium.com/max/1200/0*m_kXt3uqZH9e7H4w.png>

**Token Embedding**

```
input_ids = torch.LongTensor([[31, 51, 99], 
                              [15,  5,  0]])
```
**Positional Embedding**
```
seq_length = 3
position_ids (torch.arange): tensor([0, 1, 2])
position_ids (unsqueeze): tensor([[0, 1, 2],
                                  [0, 1, 2]])
```
**Sentence Embedding**
```
token_type_ids (torch.zeros_like): tensor([[0, 0, 0],
                                           [0, 0, 0]])
```

In [0]:
    def forward(self, input_ids, token_type_ids=None):
        seq_length = input_ids.size(1)
        position_ids = torch.arange(seq_length, dtype=torch.long, device=input_ids.device)
        position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
        if token_type_ids is None:
            token_type_ids = torch.zeros_like(input_ids)

        words_embeddings = self.word_embeddings(input_ids)
        position_embeddings = self.position_embeddings(position_ids)
        token_type_embeddings = self.token_type_embeddings(token_type_ids)

        embeddings = words_embeddings + position_embeddings + token_type_embeddings
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings

# 5. BERT Self Attention

* config: Objeto con los parametros de configuracion del modelo BERT


* self.num_attention_heads: Número de attention heads para cada capa de atención en el codificador 
Transformer

```
num_attention_heads = 12
hidden_size = 768
attention_head_size = 768/12 = 64
all_head_size = 12*64 = 768
```

* nn.Linear: Aplica una transformacion lineal a la entrada

# $ Y =X * W^{T} + bias $


```
m = nn.Linear(2, 1) # 2 nodos (entrada) y 1 nodo (salida)
m.weight: tensor([[0.2683, 0.2599]])
m.bias: tensor([0.6741])

input = torch.tensor([[1.0, -1.0]]) 

output = m(input)
1.0 * 0.2683 - 1.0 * 0.2599 + 0.6741 = 0.6825
output: tensor([[0.6825]]

```

* self.query: 768 nodos (entrada) y 768 nodos (salida)
* self.key:     768 nodos (entrada) y 768 nodos (salida)
* self.value:  768 nodos (entrada) y 768 nodos (salida)


* self.dropout: Dropout de la capa

In [0]:
class BERTSelfAttention(nn.Module):
    def __init__(self, config):
        super(BERTSelfAttention, self).__init__()
        if config.hidden_size % config.num_attention_heads != 0:
            raise ValueError(
                "The hidden size (%d) is not a multiple of the number of attention "
                "heads (%d)" % (config.hidden_size, config.num_attention_heads))
        
        self.num_attention_heads = config.num_attention_heads
        self.attention_head_size = int(config.hidden_size / config.num_attention_heads)
        self.all_head_size = self.num_attention_heads * self.attention_head_size

        self.query = nn.Linear(config.hidden_size, self.all_head_size)
        self.key =   nn.Linear(config.hidden_size, self.all_head_size)
        self.value = nn.Linear(config.hidden_size, self.all_head_size)

        self.dropout = nn.Dropout(config.attention_probs_dropout_prob)

# 5.1. Transpose for scores

* x: 

In [0]:
    def transpose_for_scores(self, x):
        new_x_shape = x.size()[:-1] + (self.num_attention_heads, self.attention_head_size)
        x = x.view(*new_x_shape)
        return x.permute(0, 2, 1, 3)

# 5.2. Forward

* hidden_states: Entrada de las capas de transformacion lineal, vector de tamaño 768
* attention_mask: 

* mixed_query_layer: Salida de las capas de transformacion lineal, vector de tamaño 768
* mixed_key_layer: Salida de las capas de transformacion lineal, vector de tamaño 768
* mixed_value_layer: Salida de las capas de transformacion lineal, vector de tamaño 768

* query_layer: 
* key_layer: 
* value_layer: 
* attention_scores: 
* attention_probs: 
* context_layer: 

In [0]:
    def forward(self, hidden_states, attention_mask):
        mixed_query_layer = self.query(hidden_states)
        mixed_key_layer = self.key(hidden_states)
        mixed_value_layer = self.value(hidden_states)

        query_layer = self.transpose_for_scores(mixed_query_layer)
        key_layer = self.transpose_for_scores(mixed_key_layer)
        value_layer = self.transpose_for_scores(mixed_value_layer)

        # Take the dot product between "query" and "key" to get the raw attention scores.
        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))
        attention_scores = attention_scores / math.sqrt(self.attention_head_size)
        # Apply the attention mask is (precomputed for all layers in BertModel forward() function)
        attention_scores = attention_scores + attention_mask

        # Normalize the attention scores to probabilities.
        attention_probs = nn.Softmax(dim=-1)(attention_scores)

        # This is actually dropping out entire tokens to attend to, which might
        # seem a bit unusual, but is taken from the original Transformer paper.
        attention_probs = self.dropout(attention_probs)

        context_layer = torch.matmul(attention_probs, value_layer)
        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[:-2] + (self.all_head_size,)
        context_layer = context_layer.view(*new_context_layer_shape)
        return context_layer

# 6. BERT Self Output

* config: 

* self.dense: 
* self.LayerNorm: 
* self.dropout: 

In [0]:
class BERTSelfOutput(nn.Module):
    def __init__(self, config):
        super(BERTSelfOutput, self).__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.LayerNorm = BERTLayerNorm(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

# 6.1. Forward

* hidden_states: 
* input_tensor: 

* self.dense: 
* self.dropout: 
* self.LayerNorm: 

In [0]:
    def forward(self, hidden_states, input_tensor):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.LayerNorm(hidden_states + input_tensor)
        return hidden_states

# 7. BERT Attention

* config: 

* self.output: 

In [0]:
class BERTAttention(nn.Module):
    def __init__(self, config):
        super(BERTAttention, self).__init__()
        self.self = BERTSelfAttention(config)
        self.output = BERTSelfOutput(config)

# 7.1. Forward

* input_tensor: 
* attention_mask: 

* attention_output: 

In [0]:
    def forward(self, input_tensor, attention_mask):
        self_output = self.self(input_tensor, attention_mask)
        attention_output = self.output(self_output, input_tensor)
        return attention_output

# 8. BERT Intermediate

* config: 

* self.dense: 
* self.intermediate_act_fn: 

In [0]:
class BERTIntermediate(nn.Module):
    def __init__(self, config):
        super(BERTIntermediate, self).__init__()
        self.dense = nn.Linear(config.hidden_size, config.intermediate_size)
        self.intermediate_act_fn = gelu

# 8.1. Forward

* hidden_states: 

In [0]:
    def forward(self, hidden_states):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.intermediate_act_fn(hidden_states)
        return hidden_states

# 9. BERT Output

* config: 

* self.dense: 
* self.LayerNorm: 
* self.dropout: 

In [0]:
class BERTOutput(nn.Module):
    def __init__(self, config):
        super(BERTOutput, self).__init__()
        self.dense = nn.Linear(config.intermediate_size, config.hidden_size)
        self.LayerNorm = BERTLayerNorm(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

# 9.1. Forward

* hidden_states: 
* input_tensor: 

* self.dense: 
* self.dropout: 
* self.LayerNorm: 

In [0]:
    def forward(self, hidden_states, input_tensor):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.LayerNorm(hidden_states + input_tensor)
        return hidden_states

# 10. BERT Layer

* config: 

* self.attention: 
* self.intermediate: 
* self.output: 

In [0]:
class BERTLayer(nn.Module):
    def __init__(self, config):
        super(BERTLayer, self).__init__()
        self.attention = BERTAttention(config)
        self.intermediate = BERTIntermediate(config)
        self.output = BERTOutput(config)

# 10.1. Forward

* hidden_states: 
* attention_mask: 

* attention_output: 
* intermediate_output: 
* layer_output: 

In [0]:
    def forward(self, hidden_states, attention_mask):
        attention_output = self.attention(hidden_states, attention_mask)
        intermediate_output = self.intermediate(attention_output)
        layer_output = self.output(intermediate_output, attention_output)
        return layer_output

# 11. BERT Encoder

* config: 

* layer: 
* nn.ModuleList: 

In [0]:
class BERTEncoder(nn.Module):
    def __init__(self, config):
        super(BERTEncoder, self).__init__()
        layer = BERTLayer(config)
        self.layer = nn.ModuleList([copy.deepcopy(layer) for _ in range(config.num_hidden_layers)])    

# 11.1. Forward

* hidden_states: 
* attention_mask: 

In [0]:
    def forward(self, hidden_states, attention_mask):
        all_encoder_layers = []
        for layer_module in self.layer:
            hidden_states = layer_module(hidden_states, attention_mask)
            all_encoder_layers.append(hidden_states)
        return all_encoder_layers

# 12. BERT Pooler

* config: 

* self.dense: 
* self.activation: 

In [0]:
class BERTPooler(nn.Module):
    def __init__(self, config):
        super(BERTPooler, self).__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.activation = nn.Tanh()

# 12.1. Forward

* hidden_states: 

* first_token_tensor: 
* pooled_output: 

In [0]:
    def forward(self, hidden_states):
        # We "pool" the model by simply taking the hidden state corresponding
        # to the first token.
        first_token_tensor = hidden_states[:, 0]
        #return first_token_tensor
        pooled_output = self.dense(first_token_tensor)
        pooled_output = self.activation(pooled_output)
        return pooled_output

# 13. BERT Model

* config: 

* self.embeddings: 
* self.encoder: 
* self.pooler: 

In [0]:
class BertModel(nn.Module):
    """BERT model ("Bidirectional Embedding Representations from a Transformer").

    Example usage:
    ```python
    # Already been converted into WordPiece token ids
    input_ids = torch.LongTensor([[31, 51, 99], [15, 5, 0]])
    input_mask = torch.LongTensor([[1, 1, 1], [1, 1, 0]])
    token_type_ids = torch.LongTensor([[0, 0, 1], [0, 2, 0]])

    config = modeling.BertConfig(vocab_size=32000, hidden_size=512,
        num_hidden_layers=8, num_attention_heads=6, intermediate_size=1024)

    model = modeling.BertModel(config=config)
    all_encoder_layers, pooled_output = model(input_ids, token_type_ids, input_mask)
    ```
    """
    def __init__(self, config: BertConfig):
        """Constructor for BertModel.

        Args:
            config: `BertConfig` instance.
        """
        super(BertModel, self).__init__()
        self.embeddings = BERTEmbeddings(config)
        self.encoder = BERTEncoder(config)
        self.pooler = BERTPooler(config)

# 13.1. Forward

* input_ids: 
* token_type_ids: 
* attention_mask: 

* extended_attention_mask: 
* embedding_output: 
* all_encoder_layers: 
* sequence_output: 
* pooled_output: 

In [0]:
    def forward(self, input_ids, token_type_ids=None, attention_mask=None):
        if attention_mask is None:
            attention_mask = torch.ones_like(input_ids)
        if token_type_ids is None:
            token_type_ids = torch.zeros_like(input_ids)

        # We create a 3D attention mask from a 2D tensor mask.
        # Sizes are [batch_size, 1, 1, from_seq_length]
        # So we can broadcast to [batch_size, num_heads, to_seq_length, from_seq_length]
        # this attention mask is more simple than the triangular masking of causal attention
        # used in OpenAI GPT, we just need to prepare the broadcast dimension here.
        extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)

        # Since attention_mask is 1.0 for positions we want to attend and 0.0 for
        # masked positions, this operation will create a tensor which is 0.0 for
        # positions we want to attend and -10000.0 for masked positions.
        # Since we are adding it to the raw scores before the softmax, this is
        # effectively the same as removing these entirely.
        extended_attention_mask = extended_attention_mask.float()
        extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0

        embedding_output = self.embeddings(input_ids, token_type_ids)
        all_encoder_layers = self.encoder(embedding_output, extended_attention_mask)
        sequence_output = all_encoder_layers[-1]
        pooled_output = self.pooler(sequence_output)
        return all_encoder_layers, pooled_output

# 14. BERT for Sequence Classification

* config: 
* num_labels: 

* self.bert: 
* self.dropout: 
* self.classifier: 

In [0]:
class BertForSequenceClassification(nn.Module):
    """BERT model for classification.
    This module is composed of the BERT model with a linear layer on top of
    the pooled output.

    Example usage:
    ```python
    # Already been converted into WordPiece token ids
    input_ids = torch.LongTensor([[31, 51, 99], [15, 5, 0]])
    input_mask = torch.LongTensor([[1, 1, 1], [1, 1, 0]])
    token_type_ids = torch.LongTensor([[0, 0, 1], [0, 2, 0]])

    config = BertConfig(vocab_size=32000, hidden_size=512,
        num_hidden_layers=8, num_attention_heads=6, intermediate_size=1024)

    num_labels = 2

    model = BertForSequenceClassification(config, num_labels)
    logits = model(input_ids, token_type_ids, input_mask)
    ```
    """
    def __init__(self, config, num_labels):
        super(BertForSequenceClassification, self).__init__()
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, num_labels)

        def init_weights(module):
            if isinstance(module, (nn.Linear, nn.Embedding)):
                # Slightly different from the TF version which uses truncated_normal for initialization
                # cf https://github.com/pytorch/pytorch/pull/5617
                module.weight.data.normal_(mean=0.0, std=config.initializer_range)
            elif isinstance(module, BERTLayerNorm):
                module.beta.data.normal_(mean=0.0, std=config.initializer_range)
                module.gamma.data.normal_(mean=0.0, std=config.initializer_range)
            if isinstance(module, nn.Linear):
                module.bias.data.zero_()
        self.apply(init_weights)

# 14.1. Forward

* input_ids: 
* token_type_ids: 
* attention_mask: 
* labels: 

* pooled_output: 
* logits: 
* loss

In [0]:
    def forward(self, input_ids, token_type_ids, attention_mask, labels=None):
        _, pooled_output = self.bert(input_ids, token_type_ids, attention_mask)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        if labels is not None:
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(logits, labels)
            return loss, logits
        else:
            return logits

# 15. BERT for Question Answering

* config: 

* self.bert: 
* self.qa_outputs: 

In [0]:
class BertForQuestionAnswering(nn.Module):
    """BERT model for Question Answering (span extraction).
    This module is composed of the BERT model with a linear layer on top of
    the sequence output that computes start_logits and end_logits

    Example usage:
    ```python
    # Already been converted into WordPiece token ids
    input_ids = torch.LongTensor([[31, 51, 99], [15, 5, 0]])
    input_mask = torch.LongTensor([[1, 1, 1], [1, 1, 0]])
    token_type_ids = torch.LongTensor([[0, 0, 1], [0, 2, 0]])

    config = BertConfig(vocab_size=32000, hidden_size=512,
        num_hidden_layers=8, num_attention_heads=6, intermediate_size=1024)

    model = BertForQuestionAnswering(config)
    start_logits, end_logits = model(input_ids, token_type_ids, input_mask)
    ```
    """
    def __init__(self, config):
        super(BertForQuestionAnswering, self).__init__()
        self.bert = BertModel(config)
        # TODO check with Google if it's normal there is no dropout on the token classifier of SQuAD in the TF version
        # self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.qa_outputs = nn.Linear(config.hidden_size, 2)

        def init_weights(module):
            if isinstance(module, (nn.Linear, nn.Embedding)):
                # Slightly different from the TF version which uses truncated_normal for initialization
                # cf https://github.com/pytorch/pytorch/pull/5617
                module.weight.data.normal_(mean=0.0, std=config.initializer_range)
            elif isinstance(module, BERTLayerNorm):
                module.beta.data.normal_(mean=0.0, std=config.initializer_range)
                module.gamma.data.normal_(mean=0.0, std=config.initializer_range)
            if isinstance(module, nn.Linear):
                module.bias.data.zero_()
        self.apply(init_weights)

# 15.1. Forward

* input_ids: 
* token_type_ids: 
* attention_mask: 
* start_positions: 
* end_positions: 

* all_encoder_layers: 
* sequence_output: 
* logits: 
* start_logits: 
* end_logits: 

In [0]:
    def forward(self, input_ids, token_type_ids, attention_mask, start_positions=None, end_positions=None):
        all_encoder_layers, _ = self.bert(input_ids, token_type_ids, attention_mask)
        sequence_output = all_encoder_layers[-1]
        logits = self.qa_outputs(sequence_output)
        start_logits, end_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)

        if start_positions is not None and end_positions is not None:
            # If we are on multi-GPU, split add a dimension - if not this is a no-op
            start_positions = start_positions.squeeze(-1)
            end_positions = end_positions.squeeze(-1)
            # sometimes the start/end positions are outside our model inputs, we ignore these terms
            ignored_index = start_logits.size(1)
            start_positions.clamp_(0, ignored_index)
            end_positions.clamp_(0, ignored_index)

            loss_fct = CrossEntropyLoss(ignore_index=ignored_index)
            start_loss = loss_fct(start_logits, start_positions)
            end_loss = loss_fct(end_logits, end_positions)
            total_loss = (start_loss + end_loss) / 2
            return total_loss
        else:
            return start_logits, end_logits